In [1]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 5.2 MB/s 


In [2]:
!pip install keras

In [3]:
!pip install nltk

In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
!pip install tflearn

     |████████████████████████████████| 107 kB 5.0 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=b033029a72fb4aa419f7955526542447ed58d7debe5ebd246bae1a99f5b67375
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [8]:
import json
import pickle

In [10]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [11]:
# import intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [12]:
intents

{'intents': [{'context_set': [''],
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!', 'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context_set': [''],
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me bot.', "I'm bot!", "I'm bot."],
   'tag': 'name'},
  {'context_set': [''],
   'patterns': ['i need help',
    'help me',
    'tell me',
    'i have some problem',
    'problem',
    'can you help me?'],
   'responses': ['yes i can help you',
    'tell me what can i do for you',
    'what do you want me to do',
    'how can i help you',
    'what kind of help do you need',
    'tell me'],
   'tag': 'problem_solving'},
  {'context_set': [''],
   'patterns': ['yellow skin',
    'yellowish white part of eye',
    'itching of the skin',
    'light colored stools',
    'dark colored urine'],
   'responses': ['you might be 

In [13]:
words=[]
classes = []
documents = []
ignore = ['?']

In [14]:
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [15]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

In [16]:
# remove duplicate classes
classes = sorted(list(set(classes)))

In [17]:
print(len(documents), "documents")

244 documents


In [18]:
print(len(classes), "classes", classes)

37 classes ['COVID-19', 'Dengue', 'Ebola', 'Hyperglycaemia', 'Hypoglycaemia', 'alcohol-related liver disease', 'allergy', 'anal cancer', 'appendicitis', 'asthma', 'bladder cancer', 'bladder problems', 'breast problems', 'chickenpox', 'common cold', 'dehydration', 'diabetic retinopathy', 'diarrhoea', 'eating or weight problems', 'emotional problems', 'eye cancer', 'fibromyalgia', 'glandular fever', 'greeting', 'hay fever', 'headache problems', 'hepatitis_B', 'influenza', 'jaundice', 'kidney cancer', 'lung problems', 'name', 'problem_solving', 'reproductive health problems', 'stroke', 'type 1 diabetes', 'type 2 diabetes']


In [19]:
print(len(words), "unique stemmed words", words)

325 unique stemmed words ["'s", '(', ')', ',', '.', '100.4f', '24', '3', '37.8c', '38c', 'a', 'abdom', 'abdomin', 'abov', 'abus', 'ach', 'act', 'alcohol', 'amount', 'an', 'anaem', 'and', 'ankl', 'anxy', 'anyon', 'apetit', 'appetit', 'ar', 'are', 'arm', 'around', 'as', 'ascit', 'at', 'attack', 'bal', 'bas', 'bed', 'behind', 'being', 'between', 'bing', 'black', 'blad', 'blee', 'block', 'blood', 'blotchy', 'blur', 'body', 'bottom', 'bout', 'bowel', 'brain', 'brea', 'breast', 'breath', 'bronchit', 'bru', 'build-up', 'bulg', 'bulk', 'burn', 'cal', 'can', 'chang', 'chest', 'chronic', 'club', 'col', 'compuls', 'confus', 'continu', 'control', 'cough', 'crack', 'cramp', 'curv', 'cut', 'dark', 'day', 'dehydr', 'delud', 'depress', 'develop', 'diarrhoe', 'diet', 'difficul', 'difficult', 'discharg', 'discomfort', 'disinterest', 'distort', 'dizzy', 'droop', 'drowsy', 'drug', 'dry', 'due', 'dur', 'ear', 'easy', 'episod', 'excess', 'exerc', 'extrem', 'ey', 'eyelid', 'fac', 'fatigu', 'feel', 'feet', 'f

In [20]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

In [21]:
# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [22]:
# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
#creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [24]:
### CREATE VIRTUAL DISPLAY ###
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,271 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.10 [784 kB]
Fetched 784 kB in 1s (963 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.10_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.10) ...
Setting up xvfb (2:1.19.6-1ubuntu4.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [26]:
import tensorflow as tf

In [31]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [27]:
#resetting underlying graph data
tf.compat.v1.reset_default_graph()

In [32]:
#Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [33]:
# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [34]:
# Start training
model.fit(train_x, train_y, n_epoch=100, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3099  | total loss: 0.97908 | time: 0.174s
| Adam | epoch: 100 | loss: 0.97908 - acc: 0.7107 -- iter: 240/244
Training Step: 3100  | total loss: 1.02753 | time: 0.178s
| Adam | epoch: 100 | loss: 1.02753 - acc: 0.6897 -- iter: 244/244
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [35]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [36]:
#restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [37]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [38]:
# load the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [39]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [40]:
# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [41]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [42]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [43]:
response("Hi")

Hi there, how can I help?


In [44]:
response("what is your name")

You can call me bot.


In [45]:
response("coughing")

you might be suffering from influeza
